In [1]:
import pyspark

conf = pyspark.SparkConf()
conf.setMaster("spark://spark:7077") 

conf.set("spark.hadoop.fs.s3a.endpoint", 'http://s3:9000') \
    .set("spark.hadoop.fs.s3a.access.key", 'minio') \
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc = pyspark.SparkContext(conf=conf)
from pyspark.sql import SparkSession
spark = SparkSession(sc)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.databricks#dbutils-api_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3c7fd4d7-a836-49b3-9a7a-257944447bc6;1.0
	confs: [default]
	found com.databricks#dbutils-api_2.12;0.0.5 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.

In [2]:
!pip install koalas==1.8.1

     |████████████████████████████████| 1.4 MB 302 kB/s eta 0:00:01


In [3]:
import time
OBJECTURL_TEST = 's3a://test-container/playground/colors-test' + str(time.time()) + '.csv'
rdd = sc.parallelize([('Mario', 'Red'), ('Luigi', 'Green'), ('Princess', 'Pink')])
rdd.toDF(['name', 'color']).write.csv(OBJECTURL_TEST, header=True)

# Read the data back from MinIO
gnames_df = spark.read.format('csv').option('header', True) \
    .load(OBJECTURL_TEST)
gnames_df.show()

21/10/18 11:59:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
21/10/18 11:59:10 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


+--------+-----+
|    name|color|
+--------+-----+
|   Luigi|Green|
|Princess| Pink|
|   Mario|  Red|
+--------+-----+



In [4]:
DELTA_URL='s3a://test-container/playground/delts-colors-test' + str(time.time())
gnames_df.write.format("delta").save(DELTA_URL)

In [5]:
data = spark.range(0, 5)
#data.write.format("delta").save("/tmp/delta-table")

In [6]:
data.write.format("delta").save("s3a://test-container/playground/to_overide")

AnalysisException: s3a://test-container/playground/to_overide already exists.

In [7]:
data.write.format("delta").mode("overwrite").save("s3a://test-container/playground/to_overide")

In [8]:
from databricks import koalas as ks
import pandas as pd
import numpy as np

In [9]:
s = ks.Series([1, 3, 5, np.nan, 6, 8])

In [10]:
kdf = ks.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [11]:
kdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


In [12]:
dates = pd.date_range('20130101', periods=6)
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
kdf = ks.from_pandas(pdf)
print(type(kdf))
kdf

<class 'databricks.koalas.frame.DataFrame'>


,A,B,C,D
2013-01-01,-0.567488,1.246976,2.674164,-1.742257
2013-01-02,-0.248967,0.544746,-0.656479,-0.070499
2013-01-03,-2.332652,1.391693,2.880239,0.131104
2013-01-04,0.587629,0.291100,0.732094,-1.719156
2013-01-05,0.412345,-0.583493,-0.631307,0.567352
2013-01-06,1.304312,1.220635,0.154229,0.479693


In [13]:
sdf = spark.createDataFrame(pdf)
sdf.show()

+-------------------+-------------------+-------------------+--------------------+
|                  A|                  B|                  C|                   D|
+-------------------+-------------------+-------------------+--------------------+
|-0.5674881744018757| 1.2469762451373825| 2.6741639830585426| -1.7422567336917387|
|-0.2489666274140646| 0.5447464950207793|-0.6564786772071499|-0.07049947932153174|
|-2.3326518183198126| 1.3916932731147502| 2.8802387758120496| 0.13110390738721153|
| 0.5876291654742538|0.29110046741344475| 0.7320941655374227| -1.7191555218947696|
| 0.4123449678875732|-0.5834932906851374|-0.6313068997974723|  0.5673516990549402|
|  1.304312202950433| 1.2206351427425046|0.15422910056417558|  0.4796931463718705|
+-------------------+-------------------+-------------------+--------------------+



In [14]:
kdf = sdf.to_koalas()

In [15]:
sdf.collect()

[Row(A=-0.5674881744018757, B=1.2469762451373825, C=2.6741639830585426, D=-1.7422567336917387),
 Row(A=-0.2489666274140646, B=0.5447464950207793, C=-0.6564786772071499, D=-0.07049947932153174),
 Row(A=-2.3326518183198126, B=1.3916932731147502, C=2.8802387758120496, D=0.13110390738721153),
 Row(A=0.5876291654742538, B=0.29110046741344475, C=0.7320941655374227, D=-1.7191555218947696),
 Row(A=0.4123449678875732, B=-0.5834932906851374, C=-0.6313068997974723, D=0.5673516990549402),
 Row(A=1.304312202950433, B=1.2206351427425046, C=0.15422910056417558, D=0.4796931463718705)]

In [16]:
kdf.to_parquet("s3a://test-container/playground/kdf")

In [17]:
ks.read_parquet("s3a://test-container/playground/kdf").head(10)

21/10/18 12:00:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,-0.567488,1.246976,2.674164,-1.742257
1,-0.248967,0.544746,-0.656479,-0.070499
2,-2.332652,1.391693,2.880239,0.131104
3,0.587629,0.291100,0.732094,-1.719156
4,0.412345,-0.583493,-0.631307,0.567352
5,1.304312,1.220635,0.154229,0.479693


In [18]:
kdf.to_delta("s3a://test-container/playground/delta_partitioned", mode='overwrite', partition_cols=["A","B"])

In [24]:
ks.read_delta("s3a://test-container/playground/delta").head(10)

21/10/18 12:02:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,-0.711055,0.225627,1.028010,0.861587
1,-0.698692,-0.668969,0.513985,-0.506543
2,0.770722,-1.601294,0.849015,-0.238955
3,1.596659,-0.834587,-0.131964,-0.216534
4,0.680701,-1.293159,0.585404,1.583542
5,-0.689997,-1.680504,0.008660,0.245247


In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable
import shutil
import random

In [28]:
data = spark.range(8)
data = data.withColumn("value", data.id + random.randint(0, 5000))
data.write.format("delta").mode("overwrite").save("s3a://test-container/playground/delta-streaming/delta-table")

In [29]:
# Stream writes to the table
print("####### Streaming write ######")
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream\
    .format("delta")\
    .option("checkpointLocation", "s3a://test-container/playground/delta-streaming/checkpoint")\
    .start("s3a://test-container/playground/delta-streaming/delta-table2")
stream.awaitTermination(10)
stream.stop()

####### Streaming write ######


21/10/18 12:03:27 WARN TaskSetManager: Lost task 41.0 in stage 71.0 (TID 880) (192.168.0.4 executor 1): TaskKilled (Stage cancelled)
21/10/18 12:03:27 WARN TaskSetManager: Lost task 28.0 in stage 71.0 (TID 881) (192.168.0.2 executor 0): TaskKilled (Stage cancelled)


In [30]:
# Stream reads from a table
print("##### Reading from stream ######")
stream2 = spark.readStream.format("delta").load("s3a://test-container/playground/delta-streaming/delta-table2")\
    .writeStream\
    .format("console")\
    .start()
stream2.awaitTermination(10)
stream2.stop()

##### Reading from stream ######


21/10/18 12:03:32 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7effab11-6adc-4c4e-970f-8dfaaf5f2a1b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+
| id|
+---+
|  7|
|  9|
| 11|
| 13|
| 15|
| 17|
| 19|
| 21|
| 23|
| 25|
| 27|
| 29|
| 31|
| 33|
| 35|
| 37|
| 39|
| 41|
| 43|
| 45|
+---+
only showing top 20 rows



In [31]:
print("####### Streaming upgrades in update mode ########")

####### Streaming upgrades in update mode ########


In [32]:
# Function to upsert microBatchOutputDF into Delta Lake table using merge
def upsertToDelta(microBatchOutputDF, batchId):
    t = deltaTable.alias("t").merge(microBatchOutputDF.alias("s"), "s.id = t.id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

In [33]:
streamingAggregatesDF = spark.readStream.format("rate").load()\
    .withColumn("id", col("value") % 10)\
    .drop("timestamp")

# Write the output of a streaming aggregation query into Delta Lake table
deltaTable = DeltaTable.forPath(spark, "s3a://test-container/playground/delta-streaming/delta-table")
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()


#############  Original Delta Table ###############
+---+-----+
| id|value|
+---+-----+
|  4| 4953|
|  5| 4954|
|  6| 4955|
|  7| 4956|
|  0| 4949|
|  1| 4950|
|  2| 4951|
|  3| 4952|
+---+-----+



In [34]:
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")

21/10/18 12:03:53 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0de633d6-cc00-46e9-8fe3-8773d433cd5e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


########### DeltaTable after streaming upsert #########
